<a href="https://colab.research.google.com/github/kkosmi/ybigta/blob/master/DL_%EC%8B%AC%ED%99%94_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [ ]:
x_train = np.reshape(x_train, (60000, 28, 28, 1))
x_test = np.reshape(x_test, (10000, 28, 28, 1))
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1), (60000,), (10000,))

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
# 멀티 클래스 분류를 할 때는 to_categorical이 편리함
y_train.shape, y_test.shape

((60000, 10), (10000, 10))

In [ ]:
i = np.random.randint(1000)
print(y_train[i])
plt.imshow(x_train[i], cmap = 'gray')

In [ ]:
# data augmentation: 사진마다 찍는 각도가 다르고, 조명이 다르고 등등. 이런 데이터의 수를 늘려주는..?
# 좀더 general 한 특징을 학습할 수 있도록 도와줌

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    validation_split = 0.25  #검증데이터도 알아서 만들어줌
)
datagen.fit(x_train)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape = (28,28,1), kernel_size = (5,5), filters = 10, activation = 'relu'),
    tf.keras.layers.MaxPool2D(pool_size = (2,2)),
    tf.keras.layers.Conv2D(kernel_size = (3,3), filters = 20, activation = 'relu'),
    tf.keras.layers.MaxPool2D(pool_size = (2,2)),
    tf.keras.layers.Conv2D(kernel_size = (3,3), filters = 20, activation = 'relu'),
    tf.keras.layers.MaxPool2D(pool_size = (2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 50, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 10, activation= 'softmax')
])
#인풋과 아웃풋의 형태는 지켜야함. 
#convolution: 인풋은 28,28,1, kernel size = 5,5, 필터개수는 열개 즉 채널이 열개로 늘어남.
#인풋shape은 첫줄에서만 설정하면 됨.
#마지막엔 확률형태의 아웃풋을 원하니까 소프트맥스

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy']
              )

model.summary()
# none은 배치 사이즈를 의미...

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 10)        260       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 20)        1820      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 20)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 20)          3620      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 20)          0         
_________________________________________________________________
flatten (Flatten)            (None, 20)                0

In [ ]:
# training 과정
# 데이터 생성기를 사용했기 때메 fit이 아니라 fit_generator를 사용함.
# flow 라는 메소드 사용: train의 레이블까지 넣어줘야 augmentation된 데이터에 대한 레이블링까지도 
# 얘가 해줌. validationsplit = 0.25라고 설정했기 때문에
train_generator = datagen.flow(x_train, y_train, batch_size = 64, subset = 'training')
val_generator = datagen.flow(x_train,y_train,batch_size=64,subset='validation')
# 이 두개를 fit generator의 매개변수로 넘겨줌!
history = model.fit_generator(train_generator,  #input
                              epochs=10,         #5번 훈련
                              verbose=1,        #결과를 간단히 표현해주세요라는 의미
                              validation_data=val_generator # 밸리데이션 데이터 지정
                              )

Epoch 1/10
  3/704 [..............................] - ETA: 29s - loss: 0.2621 - accuracy: 0.9062

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


704/704 [==============================] - 34s 49ms/step - loss: 0.1888 - accuracy: 0.9429 - val_loss: 0.1642 - val_accuracy: 0.9503
Epoch 2/10
704/704 [==============================] - 35s 49ms/step - loss: 0.1671 - accuracy: 0.9484 - val_loss: 0.1584 - val_accuracy: 0.9525
Epoch 3/10
704/704 [==============================] - 35s 50ms/step - loss: 0.1549 - accuracy: 0.9526 - val_loss: 0.1748 - val_accuracy: 0.9443
Epoch 4/10
704/704 [==============================] - 35s 50ms/step - loss: 0.1453 - accuracy: 0.9547 - val_loss: 0.1529 - val_accuracy: 0.9543
Epoch 5/10
704/704 [==============================] - 35s 50ms/step - loss: 0.1360 - accuracy: 0.9581 - val_loss: 0.1322 - val_accuracy: 0.9601
Epoch 6/10
704/704 [==============================] - 35s 50ms/step - loss: 0.1259 - accuracy: 0.9607 - val_loss: 0.1391 - val_accuracy: 0.9586
Epoch 7/10
704/704 [==============================] - 35s 50ms/step - loss: 0.1235 - accuracy: 0.9621 - val_loss: 0.1270 - val_accuracy: 0.9623
Epo

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'test loss: {loss} / test accuracy : {accuracy}')

313/313 [==============================] - 2s 7ms/step - loss: 0.0578 - accuracy: 0.9812
test loss: 0.057759106159210205 / test accuracy : 0.9811999797821045


In [ ]:
i = np.random.randint(1000)
y_pred = model(np.array([x_test[i]]))
print(f'test value = {np.argmax(y_test[i])} / pred value = {np.argmax(y_pred)}')

test value = 7 / pred value = 7
